In [1]:
signal_image_folder_path = "<PATH_TO_SPECTROGRAM_IMAGES>"
soi_path = "<PATH_TO_SPECTROGRAM_IMAGES_MARKED_WITH_SIGNAL_OF_INTEREST>"

In [2]:
import os
from torchsig.image_datasets.datasets.synthetic_signals import GeneratorFunctionDataset, rectangle_signal_generator_function, tone_generator_function, repeated_signal_generator_function
from torchsig.image_datasets.datasets.file_loading_datasets import SOIExtractorDataset, LazyImageDirectoryDataset
from torchsig.image_datasets.datasets.composites import YOLOImageCompositeDataset, CombineDataset
from torchsig.image_datasets.transforms.impairments import BlurTransform, RandomGaussianNoiseTransform, RandomImageResizeTransform, RandomRippleNoiseTransform, ScaleTransform, scale_dynamic_range, normalize_image

In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch

def plot_boxes_on_image(image, labels):
    image = image.numpy()
    image = (np.stack([image[0,:,:]]*3).transpose(1,2,0)*255).astype(np.uint8)
    for label in labels:
        cid, cx, cy, w, h = label
        img_h, img_w = image.shape[:2]
        x1 = int((cx - w/2)*img_w)
        x2 = int((cx + w/2)*img_w)
        y1 = int((cy - h/2)*img_h)
        y2 = int((cy + h/2)*img_h)
        image = cv2.rectangle(image.copy(), (x1, y1), (x2, y2), color=(255,0,0), thickness=1)
    plt.imshow(image)

In [4]:
modulations_paths = [signal_image_folder_path + fpath + "/" for fpath in os.listdir(signal_image_folder_path)]
image_datasets = []
for path in modulations_paths:
    image_datasets += [LazyImageDirectoryDataset(path, 0, read_black_hot=True, transforms = [normalize_image, lambda x: x**30, normalize_image, lambda x: x[0,(x.sum(axis=2)>1)[0]][None,:,:]**(1/10), normalize_image, scale_dynamic_range])]

FileNotFoundError: [Errno 2] No such file or directory: '<PATH_TO_SPECTROGRAM_IMAGES>'

In [ ]:
spectrogram_size = (1,2048,2048)

ripple_transform = RandomRippleNoiseTransform((0.1,0.3), num_emitors=8, base_freq=3)

tone_transforms = []
tone_transforms += [BlurTransform(strength=0.3, blur_shape=(2,1))]
tone_transforms += [ripple_transform]
tone_dataset = GeneratorFunctionDataset(tone_generator_function(spectrogram_size[-1], max_height=40), 0, transforms=tone_transforms)

signal_transforms = [normalize_image]
signal_transforms += [RandomImageResizeTransform([0.95,3])]
signal_transforms += [BlurTransform(strength=1, blur_shape=(5,1))]
signal_transforms += [ripple_transform]
image_ds = CombineDataset(image_datasets, transforms = signal_transforms)


soi_transforms = [normalize_image]
soi_transforms += [RandomGaussianNoiseTransform(mean=0, range=(0.001,0.002))]
soi_transforms += [RandomRippleNoiseTransform((0.01,0.03), num_emitors=8, base_freq=3)]
soi_dataset = SOIExtractorDataset(soi_path,1, transforms=soi_transforms,read_black_hot=True, filter_strength=50)



repeat_image_ds = CombineDataset(image_datasets, transforms = [RandomImageResizeTransform([0.3,1.5])])
repeater_transforms = [BlurTransform(strength=1, blur_shape=(5,1)), ripple_transform]
repeater_signal_dataset = GeneratorFunctionDataset(repeated_signal_generator_function(lambda: repeat_image_ds[0][0], min_gap=10, max_gap=40, min_repeats=3, max_repeats=10), 0, transforms=repeater_transforms)

composite_transforms = []
composite_transforms += [normalize_image] # inf norm
composite_transforms += [RandomRippleNoiseTransform((0.3,0.5), num_emitors=8, base_freq=1)] #light but noticeable ripple in the full image;
composite_transforms += [RandomGaussianNoiseTransform(mean=0, range=(0.3,0.5))]
composite_transforms += [scale_dynamic_range] #adjusts the noise floor in each vertical column
composite_transforms += [normalize_image] # inf norm

composite_spectrogram_dataset = YOLOImageCompositeDataset(spectrogram_size, transforms=composite_transforms, dataset_size=250000)
composite_spectrogram_dataset.add_component(tone_dataset, min_to_add=0, max_to_add=2)
composite_spectrogram_dataset.add_component(image_ds, min_to_add=0, max_to_add=3)
composite_spectrogram_dataset.add_component(soi_dataset, min_to_add=0, max_to_add=2)
composite_spectrogram_dataset.add_component(repeater_signal_dataset, min_to_add=0, max_to_add=2)


In [ ]:
image, labels = composite_spectrogram_dataset[0]
plt.rcParams["figure.figsize"] = (4,4)
plt.rcParams["figure.figsize"] = (10,10)
plot_boxes_on_image(1 - image, labels)

In [ ]:
plt.rcParams["figure.figsize"] = (10,10)
plt.imshow(image[0])